In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

from scipy import interpolate

These variables may change:

In [34]:
# basedir="/mnt/adata9/processing/"
basedir = "/home/acumpeli/adata_laptop/mnt/adata9/processing/"

animal_name='JC283'
date='20220910'

# mbasedir="/mnt/adata9/merged/m"+animal_name+'-'+date+'/'
mbasedir="/home/acumpeli/adata_laptop/mnt/adata9/merged/m"+animal_name+'-'+date+'/'

num_tetrodes=25
last_pfc_left=8
last_pfc_right=15

session_idx=[[1],[2,3,4],[5],[6],[7]]

# Possible brain regions and cell types
brain_regions = ['1', 'p', 'r', 'o', 'c']
cell_types = ['b', 'p']

These variables do not change:

In [18]:
session_names=['presleep','training1','intersleep','training2','postsleep']

sample_rate_res_old=24000

sample_rate_whl=39.0625
sample_rate_res=20000

downsample_res=sample_rate_res/sample_rate_res_old

cell_ind=2

In [50]:
# If the merged animal directory does not exist, create it
if not os.path.isdir(mbasedir):
        os.makedirs(mbasedir)

# Create a directory for this session
mfolder=basedir+animal_name+'/m'+animal_name+'-'+date+'/'

# Initialize the output res, clu, and des variables
res=np.zeros([0])
clu=np.zeros([0])
des=[]

des_full=[]


for tet_i in range(num_tetrodes):
    print("Processing tet"+str(tet_i))
    folder=basedir+animal_name+'/'+date+'/sorted/tet'+str(tet_i)+'/phy_export/'
    
    # If tetrode folder exists, continue:
    if os.path.isdir(folder)==True:
        
        # Load phy spike clusters and times
        spike_clusters=np.load(folder+'spike_clusters.npy')
        spike_times=np.load(folder+'spike_times.npy')

        # Load information about clusters (whether good, noise, or mua)
        good_noise=pd.read_csv(folder+'cluster_group.tsv',sep='\t').to_numpy()

        good_ind=np.zeros([0])
        mua_ind=np.zeros([0])

        # Iterate over clusters and keep good and mua clusters
        for i in range(good_noise.shape[0]):
            if good_noise[i,1]=='good' or good_noise[i,1]=='mua':
                neurontype=pd.read_csv(folder+'cluster_neurontype.tsv',sep='\t').to_numpy()
                good_ind=np.append(good_ind,good_noise[i,0])
                if good_noise[i,1]=='mua':
                    mua_ind=np.append(mua_ind,1)
                else:
                    mua_ind=np.append(mua_ind,0)

        # For good clusters, export neuron type label to des file and brain area to des_full
        for cell_i in range(len(good_ind)):
            cluster_idx = np.where(neurontype[:,0] == good_ind[cell_i])
            cluster_label=neurontype[cluster_idx,1][0][0]
            cell_type = cluster_label[0]
            
            # Make sure the cell label makes sense and append to des
            if (
                cluster_label[0] in cell_types
                and cluster_label[1] in brain_regions
                and len(cluster_label)==2
                ):
                
                des.append(neurontype_i)
                
                # Append to des_full based on brain region
                if tet_i<last_pfc_left:
                    des_full.append('pfc_left')
                elif tet_i<last_pfc_right:
                    des_full.append('pfc_right')
                else:
                    des_full.append('hpc_right')
                    
            else:
                print('Check label for cluster', cell_i)
                break

            res_t=spike_times[spike_clusters==good_ind[cell_i]]
            clu_t=cell_ind*np.ones(len(res_t))

            if mua_ind[cell_i]==1:
                clu_t=np.ones(len(clu_t))

            res=np.append(res,res_t)
            clu=np.append(clu,clu_t)

            cell_ind=cell_ind+1

        else:
            print('Tetrode folder missing')


Processing tet0
Processing tet1
Processing tet2
Processing tet3
Tetrode folder missing
Processing tet4
Processing tet5
Processing tet6
Processing tet7
Processing tet8
Processing tet9
Processing tet10
Processing tet11
Processing tet12
Processing tet13
Processing tet14
Processing tet15
Processing tet16
Processing tet17
Processing tet18
Tetrode folder missing
Processing tet19
Processing tet20
Processing tet21
Processing tet22
Processing tet23
Processing tet24


In [ ]:
# whl new creation

session_shift=np.loadtxt(basedir+animal_name+'/'+date+'/'+'session_shifts.txt')
session_shift=np.append([0],session_shift)




df = pd.read_csv(basedir+animal_name+'/'+date+'/'+date+'.whl',
                                        sep=' ', header=None, names=['dim1', 'dim2', 'dim1_2', 'dim2_2', 'timestamp','valid'])

whl_old=pd.DataFrame(df).to_numpy()
length_time_whl=whl_old.shape[0]/50/60/60

len_session=session_shift[-1]
length_time=len_session/sample_rate_res_old/60/60

data_points_len=(len_session)/whl_old.shape[0]
sample_rate_whl_old=data_points_len/sample_rate_res_old


sample_rate_whl_old=1/sample_rate_whl_old



x_old=np.linspace(0,len_session,whl_old.shape[0])
x_new=np.linspace(0,len_session,int(whl_old.shape[0]/sample_rate_whl_old*sample_rate_whl))

whl_new=np.zeros([len(x_new),2])

#dim1
f = interpolate.interp1d(x_old, whl_old[:,0])
y_new=f(x_new)
whl_new[:,0] = f(x_new)
#dim2
f = interpolate.interp1d(x_old, whl_old[:,1])
whl_new[:,1] = f(x_new)

#add -1 for missing
index_bad_new=np.zeros(len(x_new))
for i in range(whl_old.shape[0]):
    if whl_old[i,0]==1023:
        i_new=int(i/sample_rate_whl_old*sample_rate_whl)
        if i_new+1>len(index_bad_new)-1:
            index_bad_new[i_new-1:i_new]=1
        else:
            index_bad_new[i_new-2:i_new+2]=1


whl_new[index_bad_new>0,:]=1023


#np.savetxt(mfolder+animal_name+'-'+date+'_'+str(session_i)+'.whl', whl_new.astype(int), fmt='%i')



sort_arg=np.argsort(res)

res=res[sort_arg]
clu=clu[sort_arg]

res_down=res*downsample_res
session_shift_down=session_shift*downsample_res

##########################
print(session_shift_down)

for i in range(len(session_idx)):

    start_cut=session_idx[i][0]-1
    end_cut=session_idx[i][-1]
    
    ##############################################################
    print(start_cut, end_cut)
    print(session_idx)
    
    index1=res_down<session_shift_down[end_cut]
    index2=res_down>session_shift_down[start_cut]
    #test=np.logical_and(index1,index2)
    clu_temp=clu[np.logical_and(res_down<session_shift_down[end_cut],res_down>session_shift_down[start_cut])]
    res_temp=res_down[np.logical_and(res_down<session_shift_down[end_cut],res_down>session_shift_down[start_cut])]

    res_temp=res_temp-session_shift_down[start_cut]
    clu_temp = np.insert(clu_temp, 0, cell_ind, axis=0)


    np.savetxt(mbasedir+animal_name+'-'+date+'_'+session_names[i]+'.res', res_temp.astype(int), fmt='%i')
    np.savetxt(mbasedir+animal_name+'-'+date+'_'+session_names[i]+'.clu', clu_temp.astype(int), fmt='%i')

    start_whl=int(session_shift_down[start_cut]/sample_rate_res*sample_rate_whl)
    end_whl=int(session_shift_down[end_cut]/sample_rate_res*sample_rate_whl)
    whl_temp=whl_new[start_whl:end_whl]

    np.savetxt(mbasedir+animal_name+'-'+date+'_'+session_names[i]+'.whl', whl_temp.astype(int), fmt='%i')

    plt.plot(whl_temp[:,0],whl_temp[:,1],'*')
    plt.xlim([0,200])
    plt.ylim([0,200])
    plt.show()



with open(mbasedir+animal_name+'-'+date+'.des', 'w') as fp:
    fp.write('\n'.join(des))

with open(mbasedir+animal_name+'-'+date+'.des_full', 'w') as fp:
    fp.write('\n'.join(des_full))
